# EURO-LABS 2024: Advanced Training School on Operation of Accelerators
*T. Prebibaj, F. Asvesta

---

# PSB chromaticity correction calculation using Xsuite

---

In [32]:
# Import xsuite modules and other libraries
import xtrack as xt
import xpart as xp

from cpymad.madx import Madx
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# The MAD-X sequence is at the psb/psb.seq file. The aperture of each element is defined at psb/psb_aperture.dbx
mad = Madx()
mad.options.echo = False # to not print the stdout of MAD-X  
mad.chdir('psb')
mad.call('psb_flat_bottom.madx') # loads the PSB sequence, aperture, etc. in MAD-X
line= xt.Line.from_madx_sequence(mad.sequence['psb1'],
                                 deferred_expressions=True,
                                 allow_thick=True)
line.twiss_default['method'] = '4d' # to ignore the longitudinal plane


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.08.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2022.02.25               +
  + Execution date: 2024.05.29 16:32:41      +
  ++++++++++++++++++++++++++++++++++++++++++++
++++++ info: seqedit - number of elements installed:  0
++++++ info: seqedit - number of elements moved:      0
++++++ info: seqedit - number of elements removed:    0
++++++ info: seqedit - number of elements replaced:   0
++++++ info: seqedit - number of elements installed:  1
++++++ info: seqedit - number of elements moved:      0
++++++ info: seqedit - number of elements removed:    0
++++++ info: seqedit - number of elements replaced:   0


Converting sequence "psb1":   0%|          | 0/530 [00:00<?, ?it/s]

In [37]:
# We need to add a reference particle for the twiss calculation (mass, energy)
line.particle_ref=xp.Particles(mass0=xp.PROTON_MASS_EV,gamma0=mad.sequence.psb1.beam.gamma)
print('Reference particle added at gamma0=%s.'%(mad.sequence.psb1.beam.gamma))

Reference particle added at gamma0=1.1705262269290748.


In [38]:
# Tune matching
line.vars['kbr1xnoh0'] = 0.0
Qx0 = 4.403
Qy0 = 4.453
line.match(
      vary=[
            xt.Vary('kbrqf', step=1e-8),
            xt.Vary('kbrqd', step=1e-8),
      ],
      targets = [
                  xt.Target('qx', Qx0, tol=1e-5),
                  xt.Target('qy', Qy0, tol=1e-5)
      ]
)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Matching: model call n. 4       



In [39]:
twiss = line.twiss()
twiss_df = twiss.to_pandas()

In [42]:
print('Natural machine:')
print('(Q_x, Q_y) = (%s, %s)'%(twiss.qx,twiss.qy)) # Tunes
print("(Q'_x, Q'_y) = (%s, %s)"%(twiss.dqx,twiss.dqy)) # Chromaticity

Natural machine:
(Q_x, Q_y) = (4.402998505462361, 4.45300227179516)
(Q'_x, Q'_y) = (-3.736959295910225, -7.333920582466291)


In [43]:
# Correcting chromaticity in one plane
plane = 'y'
line.match(
    vary=[
        xt.Vary('kbr1xnoh0', step=1e-8)
    ],
    targets = [
        xt.Target('dq%s'%plane, 0.0, tol=1e-5)
    ]
)

Matching: model call n. 6       



In [44]:
twiss_after = line.twiss()

In [45]:
print('After correcting chromaticity in %s plane:'%plane)
print('(Q_x, Q_y) = (%s, %s)'%(twiss_after.qx,twiss_after.qy)) # Tunes
print("(Q'_x, Q'_y) = (%s, %s)"%(twiss_after.dqx,twiss_after.dqy)) # Chromaticity
print('Sextupoles strength needed for correction: ', line.vars['kbr1xnoh0']._value)
print('+0.3098 corresponds to 78 Amps in the real BR.XNOH0; to be verified')

After correcting chromaticity in y plane:
(Q_x, Q_y) = (4.402998505451622, 4.4530022716522195)
(Q'_x, Q'_y) = (-7.1705321095105745, 1.119992987241858e-08)
Sextupoles strength needed for correction:  0.370056607427916
+0.3098 corresponds to 78 Amps in the real BR.XNOH0; to be verified


### To try and "match" exactly what is measured in the accelerator (measured tunes + measured chromaticity)

In [46]:
# First setup the machine to the measured state
target_Qx = 4.403
target_Qy = 4.453
target_dQy = -7.0
line.match(
      vary=[
            xt.Vary('kbrqf', step=1e-8),
            xt.Vary('kbrqd', step=1e-8),
            xt.Vary('kbr1xnoh0', step=1e-8)
      ],
      targets = [
                  xt.Target('qx', Qx0, tol=1e-5),
                  xt.Target('qy', Qy0, tol=1e-5),
                  xt.Target('dq%s'%plane, target_dQy, tol=1e-5)
      ]
)
twiss1 = line.twiss()
print('Measured state:')
print('(Q_x, Q_y) = (%s, %s)'%(twiss1.qx,twiss1.qy))
print("(Q'_x, Q'_y) = (%s, %s)"%(twiss1.dqx,twiss1.dqy))
print('Sextupoles strength: ', line.vars['kbr1xnoh0']._value)
kxnoh0_1 = line.vars['kbr1xnoh0']._value

# Now correct the chromaticity
line.match(
    vary=[
        xt.Vary('kbr1xnoh0', step=1e-8)
    ],
    targets = [
        xt.Target('dq%s'%plane, 0.0, tol=1e-5)
    ]
)
twiss2 = line.twiss()
print('After correcting chromaticity:')
print('(Q_x, Q_y) = (%s, %s)'%(twiss2.qx,twiss2.qy))
print("(Q'_x, Q'_y) = (%s, %s)"%(twiss2.dqx,twiss2.dqy))
print('Sextupoles strength: ', line.vars['kbr1xnoh0']._value)
kxnoh0_2 = line.vars['kbr1xnoh0']._value

# Calculate the difference
deltak  = kxnoh0_2 - kxnoh0_1
print('The difference in the sextupole strength is: ', deltak)

Matching: model call n. 10       

Measured state:
(Q_x, Q_y) = (4.403000007486291, 4.453000000017937)
(Q'_x, Q'_y) = (-3.8932928140500422, -6.9999999994685425)
Sextupoles strength:  0.016848898998567115
Matching: model call n. 6       

After correcting chromaticity:
(Q_x, Q_y) = (4.403000007501474, 4.4529999998753)
(Q'_x, Q'_y) = (-7.170530753493054, -5.313083306646149e-08)
Sextupoles strength:  0.37005668247912993
The difference in the sextupole strength is:  0.3532077834805628
